In [57]:
import pandas as pd
import numpy as np

In [58]:
cnv_tsv = '/s/project/mll/rawdata/res_from_mll/cnv_calls.txt'

gene_coords = '/s/project/mll/drop_2023feb/processed_data/preprocess/v33b/gene_name_mapping_v33b.tsv'

In [59]:
cnv_df = pd.read_csv(cnv_tsv, sep='\t',
                    usecols=[0,1,2,5,6], names=['chrom','start','end','call','array_id'], skiprows=1)
cnv_df = cnv_df[(~cnv_df.chrom.isna())&(cnv_df.call!='0')]
cnv_df.sort_values(by=['chrom','start','end'],inplace=True)
cnv_df['geneName']=None

In [60]:
gene_df = pd.read_csv(gene_coords)
gene_df.rename(columns={'seqnames':'chrom','gene_id':'geneName','gene_name':'geneHGNC'},inplace=True)
gene_df.geneName = gene_df.geneName.apply(lambda x:x.split('.')[0])
gene_df.chrom = gene_df.chrom.str.replace('chr','')
gene_df.sort_values(by=['chrom','start','end'],inplace=True)

In [61]:
def getOverlap(a, b):
    '''
    Returns the amount of overlap between two intervals
    '''
    #print(a,b)
    return max(0, min(a[1], b[1]) - max(a[0], b[0]))

In [62]:
#loop over copy number intervals and check if they overlap with genes

cnv_df['geneName'] = np.empty((len(cnv_df), 0)).tolist()

    
for chrom in cnv_df.chrom.unique():
    
    print(chrom)
    
    #select given chromosome
    cnv_chrom_df = cnv_df[cnv_df.chrom==chrom]
    gene_chrom_df = gene_df[gene_df.chrom==chrom] #gene coordinates in given chromosome
    
    for idx, row in cnv_chrom_df.iterrows():
        int_start, int_end = row.start, row.end
        for geneName,gene_start,gene_end in gene_chrom_df[['geneName','start','end']].values.tolist():
            if getOverlap((int_start, int_end),(gene_start,gene_end))>0:
                cnv_df.at[idx,'geneName'].append(geneName)
                
#    for idx, row in cnv_chrom_df.iterrows():
#        int_start, int_end = row.start, row.end
#        #interval end should be larger than the gene start
#        max_row_idx = np.searchsorted(gene_chrom_df.start, int_end) #gene_df should already be sorted by chromosome and by start
#        df = gene_chrom_df.iloc[:max_row_idx].sort_values(by='end')
#        #interval start should be smaller than the gene end
#        min_row_idx = np.searchsorted(df.end, int_start)
#        df = df.iloc[min_row_idx:] #now all intervals remaining in df should overlap
#        
#        sufficient_overlap = df.apply(lambda x: getOverlap((x.start,x.end),(int_start, int_end)), axis=1)>0
#        
#        cnv_df.at[idx,'geneName'] = df.loc[sufficient_overlap].geneName.values.tolist() #add the list of genes for given interval

1
10
11
12
13
14
15
16
17
18
19
2
20
21
22
3
4
5
6
7
8
9
X
Y


In [63]:
cnv_df = cnv_df.explode('geneName')

In [64]:
cnv_df = cnv_df[~cnv_df.geneName.isna()]

In [66]:
cnv_df.to_csv('/s/project/mll/sergey/MLL_data/processed/cnv_mll_gencode_gene_map_bruteforce.tsv',sep='\t')

In [67]:
len(cnv_df.drop_duplicates())

16557965